In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pygtrans

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#互译方案
from pygtrans import Translate
#这里传入的sentences是一个英文句子
def backTran(sentences):
    client = Translate()
    text1 = client.translate(sentences)

    text2 = client.translate(text1.translatedText, target='en')
    return text2.translatedText


In [ ]:
#随机删除
#这里传入的sentences是一个英文句子
def random_deletion(sentences, p):
    words = sentences.split()
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return " ".join(new_words)

#随机交换
#随机交换n次
#这里传入的sentences是一个英文句子
def random_swap(sentences, n):
    words = sentences.split()
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return " ".join(new_words)

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

In [ ]:
#同义词替换
#替换一个语句中的n个单词为其同义词
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords#引入停用词，因为对停用词进行数据增强相当于没有增强
from nltk.corpus import wordnet as wn#引入同义词
import random

stop_words=stopwords.words('english')
for w in ['!',',','.','?','-s','-ly','</s>','s']:
    stop_words.append(w)
def synonym_replacement(words):
    new_words = words.copy()
    n=len(words)/4
    random_word_list = list(set([word for word in words if word not in stop_words]))     
    random.shuffle(random_word_list)
    num_replaced = 0  
    for random_word in random_word_list:          
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)   
            new_words = [synonym if word == random_word else word for word in new_words]   
            num_replaced += 1
        if num_replaced >= n: 
            break

    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return " ".join(new_words)

#获取同义词
def get_synonyms(word):
    nearbyWordSet=wn.synsets(word)
    return nearbyWordSet[0].lemma_names()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/train_en.csv',index_col=0)
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/testA_zh_en_final.csv')

train_en = train[['title_en','assignee_en','abstract_en','label_id']]
train_en.head()

,title_en,assignee_en,abstract_en,label_id
0,"Signal transmission method, base station and u...",Huawei Technology Co. Ltd.,The invention relates to a method for transmit...,0
1,A Fault Monitoring system for 5G Communication...,China Railway 22 Bureau Group electrification ...,The invention discloses a 5G communication cab...,0
2,Test method and electronic device,"Tencent Technology (Beijing) Co., Ltd.",The invention provides a test method and an el...,0
3,Antenna azimuth adjusting device,"Wuhan Hongxin Communication Technology Co., Ltd.",An antenna azimuth adjusting device comprises ...,0
4,Optical fiber aging prediction method and device,"Xinhua San big data Technology Co., Ltd.",The application provides an optical fiber agin...,0


In [ ]:
aug_train = pd.DataFrame(columns=['title_en','assignee_en','abstract_en','label_id'])
aug_test = pd.DataFrame(columns=['title_en','assignee_en','abstract_en'])

# 互译处理，其他处理方式也一样
#aug_train["title_en"] = train_en.title_en.apply(lambda x: synonym_replacement(x))
#aug_train["assignee_en"] = train_en.assignee_en.apply(lambda x: synonym_replacement(x))
#aug_train["abstract_en"] = train_en.abstract_en.apply(lambda x: synonym_replacement(x))
aug_train["title_en"] = train_en["title_en"]
aug_train["assignee_en"] = train_en["assignee_en"]
aug_train["abstract_en"] = train_en.abstract_en.tolist()
aug_train['label_id'] = train_en['label_id']
#aug_test["title_en"] = test_en.title_en.apply(lambda x: backTran(x))
#aug_test["assignee_en"] = test_en.assignee_en.apply(lambda x: backTran(x))
#aug_test["abstract_en"] = test_en.abstract_en.apply(lambda x: backTran(x))
aug_train.at[0,'abstract_en']

'The invention relates to a method for transmitting a signal, a base station and user equipment. The time domain position relationship between the symbols of the reference signal and the symbols of the data configured for multiple user equipment in a subframe satisfies premise one and premise two. The first premise is that the resources required for the reference signal of each user equipment are included in the symbols of a plurality of reference signals, and the second premise is at least one of the following conditions: the multiple reference signals of each user equipment are arranged in the symbols of the reference signals before the symbols of the data of each user equipment, and / or in the symbols of the reference signals after the symbols of the data of each user equipment. As a result, the cost of sending reference signals is effectively saved and the requirements of resource design are met. And some or all of the user equipment can include its reference signal in the symbols

In [ ]:
test.describe()

,id,title,assignee,abstract,title_en,assignee_en,abstract_en
count,20839,20839,20839,20839,20839,20839,20839
unique,20839,20428,10836,20839,20278,10808,20839
top,003fd481e65ddc070e38ae05002e16e2,半导体结构及其形成方法,华为技术有限公司,本发明公开了一种耐磨、抗粘钢复合涂层、制备方法及应用，包括基体和基体上由内到外依次设置的过渡...,Display panel and display device,Huawei Technology Co. Ltd.,The invention discloses a wear-resistant and a...
freq,1,24,419,1,68,419,1


In [ ]:
test_en.at[0,'abstract_en']

'The invention relates to a method for transmitting a signal, a base station and user equipment. The time domain position relationship between the symbols of the reference signal and the symbols of the data configured for multiple user equipment in a subframe satisfies premise one and premise two. The first premise is that the resources required for the reference signal of each user equipment are included in the symbols of a plurality of reference signals, and the second premise is at least one of the following conditions: the multiple reference signals of each user equipment are arranged in the symbols of the reference signals before the symbols of the data of each user equipment, and / or in the symbols of the reference signals after the symbols of the data of each user equipment. As a result, the cost of sending reference signals is effectively saved and the requirements of resource design are met. And some or all of the user equipment can include its reference signal in the symbols

In [ ]:
aug_test.to_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/aug_data/test_en_backTran.csv',index=False)

In [2]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/train_en.csv',index_col=0)
aug_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/aug_data/train_en_backTran.csv')
aug_train['id'] = train['id']
aug_train['label_id'] = train['label_id']
aug_train = aug_train[['id', 'title_en', 'assignee_en', 'abstract_en','label_id']]
aug_train.head()

,id,title_en,assignee_en,abstract_en,label_id
0,538f267d2e6fba48b1286fb7f1499fe7,"Signal transmission method, base station and u...","Huawei Technologies Co., Ltd",The present invention relates to a signal tran...,0
1,635a7d4b6358b6ff24a324bb871505db,A 5G communication cable fault monitoring system,China Railway 22nd Bureau Group Electrificatio...,The invention discloses a 5G communication cab...,0
2,aaf98d6bfe1932cf1a262812ca59d1ba,Test method and electronic device,"Tencent Technology (Beijing) Co., Ltd.",The invention provides a testing method and an...,0
3,ad49c9ba6412452d9b25071af702f4fb,Antenna Azimuth Adjustment Device,"Wuhan Hongxin Communication Technology Co., Ltd.","An antenna azimuth angle adjustment device, co...",0
4,ffa2d7617b3eac3a1d7df01e5bb515a2,Optical fiber aging prediction method and device,"Xinhua Three Data Technology Co., Ltd.",The present application provides an optical fi...,0


In [3]:
train = train[['id', 'title_en', 'assignee_en', 'abstract_en','label_id']]
train.head()

,id,title_en,assignee_en,abstract_en,label_id
0,538f267d2e6fba48b1286fb7f1499fe7,"Signal transmission method, base station and u...",Huawei Technology Co. Ltd.,The invention relates to a method for transmit...,0
1,635a7d4b6358b6ff24a324bb871505db,A Fault Monitoring system for 5G Communication...,China Railway 22 Bureau Group electrification ...,The invention discloses a 5G communication cab...,0
2,aaf98d6bfe1932cf1a262812ca59d1ba,Test method and electronic device,"Tencent Technology (Beijing) Co., Ltd.",The invention provides a test method and an el...,0
3,ad49c9ba6412452d9b25071af702f4fb,Antenna azimuth adjusting device,"Wuhan Hongxin Communication Technology Co., Ltd.",An antenna azimuth adjusting device comprises ...,0
4,ffa2d7617b3eac3a1d7df01e5bb515a2,Optical fiber aging prediction method and device,"Xinhua San big data Technology Co., Ltd.",The application provides an optical fiber agin...,0


In [6]:
mix = pd.concat([train,aug_train])
mix.isnull()

,id,title_en,assignee_en,abstract_en,label_id
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
953,False,False,False,False,False
954,False,False,False,False,False
955,False,False,False,False,False
956,False,False,False,False,False


In [7]:
mix.to_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/aug_data/train_en_BackTrans_Concat.csv',index=False)